In [2]:
##########################################################
# パッケージの読み込みとエイリアスの設定
##########################################################
using Revise, LinearAlgebra, LaTeXStrings, Plots, CSV, DataFrames, JLD2, Random, Statistics, Distributions
using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
using Tensor_FixedSeedMC
import Tensor_FixedSeedMC as TCIMC

In [3]:
n = 32
a_ = -2.0
b_ = 2.0
xs = TCIMC.chebyshev_lobatto_nodes(a_, b_, n) # 1) ロバット点（昇順）
g = x -> sin(x)
y = g.(xs)                                 # 2) y = f.(xs)

33-element Vector{Float64}:
 -0.9092974268256817
 -0.9132629192266357
 -0.9246144278719843
 -0.9417215937293848
 -0.9618903686220686
 -0.9814243775069309
 -0.9957578459010482
 -0.9996931049789023
 -0.9877659459927355
 -0.954740635924686
 -0.8962052244279891
 -0.8092025046847494
 -0.6928016016971985
  ⋮
  0.8092025046847495
  0.8962052244279893
  0.954740635924686
  0.9877659459927356
  0.9996931049789023
  0.9957578459010483
  0.9814243775069309
  0.9618903686220686
  0.9417215937293848
  0.9246144278719843
  0.9132629192266356
  0.9092974268256817

In [4]:
# 3) 係数 a を行列掛け算で取得
A = TCIMC.chebyshev_coeff_matrix_from_lobatto_sorted(n)
a = A * y

33-element Vector{Float64}:
 -8.326672684688674e-17
  1.1534496155137466
  2.7755575615628914e-17
 -0.2578864989488043
  1.3877787807814457e-17
  0.014079259511743408
  3.122502256758253e-16
 -0.00034988814973718405
  3.7816971776294395e-16
  4.984686869876492e-6
  1.0755285551056204e-16
 -4.6085695246111946e-8
  3.0531133177191805e-16
  ⋮
 -5.516420653606247e-16
 -4.475586568020162e-16
  1.6826817716975029e-15
 -1.124100812432971e-15
  6.522560269672795e-16
  3.5735303605122226e-16
 -3.8163916471489756e-16
  7.216449660063518e-16
 -3.2959746043559335e-16
  1.3530843112619095e-15
  5.30825383648903e-16
  2.7755575615628914e-17

In [5]:
xs_ = collect(range(a_, stop=b_, length=100))

# --- 方法1: 事前確保して添字で代入 ---
y_hat = zeros(length(xs_))
@inbounds for (i, x) in enumerate(xs_)
    φ = TCIMC.cheb_basis_vector_weighted(a_, b_, x, n)
    y_hat[i] = dot(φ, a)               # 係数との内積で評価
end

# 参考: 真値と比較（例: f(x)=exp(x) なら）
y_true = g.(xs_)


100-element Vector{Float64}:
 -0.9092974268256817
 -0.9253647591085917
 -0.9399216514301312
 -0.952944343093062
 -0.9644115776212773
 -0.9743046374559903
 -0.9826073745077393
 -0.9893062365143401
 -0.9943902891617602
 -0.9978512339318114
 -0.9996834216475249
 -0.9998838616941024
 -0.9984522269003895
  ⋮
  0.9998838616941024
  0.9996834216475249
  0.9978512339318114
  0.9943902891617602
  0.9893062365143401
  0.9826073745077393
  0.9743046374559903
  0.9644115776212773
  0.952944343093062
  0.9399216514301312
  0.9253647591085917
  0.9092974268256817

In [6]:
@show maximum(y_true .- y_hat)

maximum(y_true .- y_hat) = 7.105427357601002e-15


7.105427357601002e-15

In [10]:
#φp = cheb_basis_vector_weighted_derivative_noU(x, n)
# 係数ベクトル a があれば、p'(x) ≈ dot(φp, a)

xs_ = collect(range(-1.0, stop=1.0, length=100))

# --- 方法1: 事前確保して添字で代入 ---
y_hat = zeros(length(xs_))
@inbounds for (i, x) in enumerate(xs_)
    φp = TCIMC.cheb_basis_vector_weighted_derivative(a_, b_, x, n)
    y_hat[i] = dot(φp, a)               # 係数との内積で評価
end

# 参考: 真値と比較（例: f(x)=exp(x) なら）
y_true = cos.(xs_)
@show maximum(y_true .- y_hat)

maximum(y_true .- y_hat) = 3.5416114485542494e-14


3.5416114485542494e-14

In [11]:
d1(S0, K::Float64, σ::Float64, r::Float64,  t::Float64) = (log(S0/K) + (r + σ^2/2) * (1.0 - t)) / (σ * (1.0 - t)^2)
d1(S0) = (log(S0/100.0) + (0.05 + 0.2^2/2) * (1.0 - 0.0)) / (0.2 * (1.0 - 0.0)^2)

d1 (generic function with 2 methods)

In [16]:
S0s_equiv = collect(range(90, stop=120.0, length=100))
n = 100
S0s_lobatto = TCIMC.chebyshev_lobatto_nodes(90.0, 120.0, n)
y_ = d1.(S0s_lobatto)        
A = TCIMC.chebyshev_coeff_matrix_from_lobatto_sorted(n)
a_ = A * y_
# --- 方法1: 事前確保して添字で代入 ---
op_hat = zeros(length(S0s_equiv))
@inbounds for (i, x) in enumerate(S0s_equiv)
    #φp = TCIMC.cheb_basis_vector_weighted_derivative(a_, b_, x, n)
    φ = TCIMC.cheb_basis_vector_weighted(90, 120, x, n)
    op_hat[i] = dot(φ, a_)               # 係数との内積で評価
end

# 参考: 真値と比較（例: f(x)=exp(x) なら）
op_true = d1.(S0s_equiv)
@show maximum(op_true .- op_hat)

maximum(op_true .- op_hat) = 7.882583474838611e-15


7.882583474838611e-15